In [ ]:
import pandas as pd

pd.set_option('display.float_format', '{:.13f}'.format)

vertical_velocity = pd.read_csv('vertical velocity at t points.csv', skiprows = 14, header= None, float_precision='round_trip')
surface_height = pd.read_csv('/content/surface height on  t cell.csv', skiprows = 14, header= None)

vertical_velocity.columns = ['DATETIME', 'TIME', 'LON', 'LAT', 'DEP','WT']

vertical_velocity = vertical_velocity.drop('DATETIME', axis=1)
vertical_velocity = vertical_velocity.drop('DEP', axis=1)



surface_height.columns = ['DATETIME', 'TIME', 'LON', 'LAT','ETA_T']

surface_height = surface_height.drop('DATETIME', axis=1)
# surface_height = surface_height.drop('DEP', axis=1)

merged_data = pd.merge(
    vertical_velocity,
    surface_height,
    on=['TIME', 'LON', 'LAT'],
    how='inner'
)

print(merged_data.head())

print(f"Shape of merged dataset: {merged_data.shape}")

# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# merged_data['WT_scaled'] = scaler.fit_transform(merged_data[['WT']])


                 TIME              LON              LAT               WT  \
0 44194.5000000000000 83.7500000000000 15.9798000000000 -0.0000033528200   
1 44194.5000000000000 81.2500000000000 16.3134000000000 -0.0000039097800   
2 44194.5000000000000 81.7500000000000 16.3134000000000  0.0000015113600   
3 44194.5000000000000 82.2500000000000 16.3134000000000 -0.0000038293300   
4 44194.5000000000000 82.7500000000000 16.3134000000000  0.0000016903300   

            ETA_T  
0 0.6024660000000  
1 0.7213450000000  
2 0.7160020000000  
3 0.6997060000000  
4 0.7058900000000  
Shape of merged dataset: (17467, 5)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


merged_data['WT'] = merged_data['WT'].replace(-1.0E+34, np.nan)

daily_means = merged_data.groupby('TIME')['WT'].mean()

for time, group in merged_data.groupby('TIME'):
    merged_data.loc[(merged_data['TIME'] == time) & (merged_data['WT'].isna()), 'WT'] = daily_means[time]



merged_data['ETA_T'] = merged_data['ETA_T'].replace(-1.0E+34, np.nan)

daily_means = merged_data.groupby('TIME')['ETA_T'].mean()

for time, group in merged_data.groupby('TIME'):
    merged_data.loc[(merged_data['TIME'] == time) & (merged_data['ETA_T'].isna()), 'ETA_T'] = daily_means[time]


print(merged_data)

                     TIME              LON              LAT               WT  \
0     44194.5000000000000 83.7500000000000 15.9798000000000 -0.0000033528200   
1     44194.5000000000000 81.2500000000000 16.3134000000000 -0.0000039097800   
2     44194.5000000000000 81.7500000000000 16.3134000000000  0.0000015113600   
3     44194.5000000000000 82.2500000000000 16.3134000000000 -0.0000038293300   
4     44194.5000000000000 82.7500000000000 16.3134000000000  0.0000016903300   
...                   ...              ...              ...              ...   
17462 44557.5000000000000 81.7500000000000 18.4590000000000 -0.0000008072294   
17463 44557.5000000000000 82.2500000000000 18.4590000000000 -0.0000008072294   
17464 44557.5000000000000 82.7500000000000 18.4590000000000 -0.0000008072294   
17465 44557.5000000000000 83.2500000000000 18.4590000000000 -0.0000008072294   
17466 44557.5000000000000 83.7500000000000 18.4590000000000 -0.0000008072294   

                ETA_T  
0     0.6024660

In [ ]:
X = merged_data[['TIME', 'LON', 'LAT', 'ETA_T']]
y = merged_data['WT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression Results:")
print(f"Mean Squared Error: {mse_lr:.6f}")
print(f"R² Score: {r2_lr:.6f}")

coefficients = pd.DataFrame({
    'Feature': ['TIME', 'LON', 'LAT', 'ETA_T'],
    'Coefficient': lr_model.coef_
})
intercept = lr_model.intercept_

print("Model Coefficients:")
print(coefficients)
print(f"Intercept: {intercept:.6f}")


Linear Regression Results:
Mean Squared Error: 0.000000
R² Score: 0.112095
Model Coefficients:
  Feature      Coefficient
0    TIME -0.0000000100124
1     LON -0.0000001757412
2     LAT  0.0000005161985
3   ETA_T -0.0000333515751
Intercept: 0.000471


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\nRandom Forest Results:")
print(f"Mean Squared Error: {mse_rf:.15f}")
print(f"R² Score: {r2_rf:.6f}")

feature_importance = pd.DataFrame({
    'Feature': ['TIME', 'LON', 'LAT', 'ETA_T'],
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance:")
print(feature_importance)



Random Forest Results:
Mean Squared Error: 0.000000000026821
R² Score: 0.679403
Feature Importance:
  Feature      Importance
0    TIME 0.5068006897585
3   ETA_T 0.3111899290940
2     LAT 0.1031979891585
1     LON 0.0788113919890
